IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M3\NO_FAULT3


In [3]:
guasto=0

VOLO m3 NO FAULT

In [4]:
rcou_m3_nofault = pd.read_csv("RCOU.csv")
rcou_m3_nofault = rcou_m3_nofault.astype("float64")
rcou_m3_nofault = rcou_m3_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m3_nofault = rcou_m3_nofault[((rcou_m3_nofault['C9'] >= potenza) & (rcou_m3_nofault['C10'] >= potenza) & (rcou_m3_nofault['C11'] >= potenza) & (rcou_m3_nofault['C12'] >= potenza) & (rcou_m3_nofault['C13'] >= potenza) & (rcou_m3_nofault['C14']>= potenza))]
rcou_m3_nofault=rcou_m3_nofault.reset_index(drop=True)
display(rcou_m3_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,85323839.0,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0
1,85345385.0,1411.0,1395.0,1437.0,1368.0,1441.0,1363.0
2,85356318.0,1376.0,1441.0,1404.0,1415.0,1366.0,1450.0
3,85367015.0,1409.0,1423.0,1421.0,1411.0,1412.0,1420.0
4,85423541.0,1523.0,1364.0,1442.0,1456.0,1515.0,1375.0
...,...,...,...,...,...,...,...
6615,162729348.0,1551.0,1519.0,1559.0,1510.0,1447.0,1613.0
6616,162740384.0,1598.0,1461.0,1520.0,1545.0,1460.0,1599.0
6617,162751646.0,1620.0,1423.0,1544.0,1513.0,1512.0,1545.0
6618,162761906.0,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0


In [5]:
min=rcou_m3_nofault['TimeUS'][0]
max=rcou_m3_nofault['TimeUS'][len(rcou_m3_nofault)-1]
print(max)
print(min)

162772643.0
85323839.0


In [6]:
xkf1_m3_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m3_nofault = xkf1_m3_nofault.astype("float64")
xkf1_m3_nofault = xkf1_m3_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_nofault = xkf1_m3_nofault[((xkf1_m3_nofault['TimeUS'] >= min) & (xkf1_m3_nofault['TimeUS'] <= max))]
xkf1_m3_nofault = xkf1_m3_nofault.reset_index(drop=True)
print(xkf1_m3_nofault)

           TimeUS  Roll  Pitch     Yaw
0      85332681.0  1.98  -0.84    3.31
1      85343445.0  1.98  -0.88    3.31
2      85353408.0  1.91  -0.83    3.33
3      85364246.0  1.93  -0.80    3.33
4      85375369.0  2.02  -0.82    3.32
...           ...   ...    ...     ...
7005  162727101.0 -1.18   1.24  358.17
7006  162738085.0 -1.06   1.13  358.20
7007  162749622.0 -0.97   1.00  358.25
7008  162759649.0 -0.91   0.94  358.31
7009  162770128.0 -0.79   0.88  358.39

[7010 rows x 4 columns]


In [7]:
att_m3_nofault = pd.read_csv("ATT.csv")
att_m3_nofault = att_m3_nofault.astype("float64")
att_m3_nofault = att_m3_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_nofault = att_m3_nofault[((att_m3_nofault['TimeUS'] >= min) & (att_m3_nofault['TimeUS'] <= max	))]
att_m3_nofault=att_m3_nofault.reset_index(drop=True)
print(att_m3_nofault)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      85334020.0     0.12  1.98      0.24  -0.84    3.31    3.31
1      85344873.0     0.12  1.98      0.25  -0.88    3.31    3.31
2      85355543.0     0.12  1.91      0.26  -0.83    3.31    3.33
3      85366178.0     0.12  1.93      0.26  -0.80    3.31    3.33
4      85377132.0     0.12  2.02      0.25  -0.82    3.31    3.32
...           ...      ...   ...       ...    ...     ...     ...
7007  162728615.0     0.96 -1.18     -0.36   1.24  359.91  358.18
7008  162739678.0     0.95 -1.06     -0.37   1.13  359.91  358.21
7009  162751006.0     0.93 -0.97     -0.38   1.00  359.91  358.26
7010  162761104.0     0.89 -0.91     -0.36   0.94  359.91  358.32
7011  162771993.0     0.83 -0.79     -0.33   0.88  359.91  358.40

[7012 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m3_nofault)>len(xkf1_m3_nofault):
    to_add=att_m3_nofault[len(xkf1_m3_nofault):]
    att_m3_nofault=att_m3_nofault[:len(xkf1_m3_nofault)]

if len(xkf1_m3_nofault)>len(att_m3_nofault):
    to_add=xkf1_m3_nofault[len(att_m3_nofault):]

for idx,i in enumerate(att_m3_nofault['Roll']):
    if(xkf1_m3_nofault['Roll'][idx] != i):
        att_m3_nofault['Roll'][idx] = ((att_m3_nofault['Roll'][idx] + xkf1_m3_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m3_nofault['Pitch']):
    if(xkf1_m3_nofault['Pitch'][idx] != i):
        att_m3_nofault['Pitch'][idx] = ((att_m3_nofault['Pitch'][idx] + xkf1_m3_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_nofault['Yaw']):
    if(xkf1_m3_nofault['Yaw'][idx] != i):
        att_m3_nofault['Yaw'][idx] = ((att_m3_nofault['Yaw'][idx] + xkf1_m3_nofault['Yaw'][idx])/2)

if not to_add.empty:
    att_m3_nofault=pd.concat([att_m3_nofault,to_add])

print(att_m3_nofault)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0      85334020.0     0.12  1.980      0.24 -0.840    3.31    3.310
1      85344873.0     0.12  1.980      0.25 -0.880    3.31    3.310
2      85355543.0     0.12  1.910      0.26 -0.830    3.31    3.330
3      85366178.0     0.12  1.930      0.26 -0.800    3.31    3.330
4      85377132.0     0.12  2.020      0.25 -0.820    3.31    3.320
...           ...      ...    ...       ...    ...     ...      ...
7007  162728615.0     0.96 -1.075     -0.36  1.120  359.91  358.215
7008  162739678.0     0.95 -0.985     -0.37  1.035  359.91  358.260
7009  162751006.0     0.93 -0.880     -0.38  0.940  359.91  358.325
7010  162761104.0     0.89 -0.910     -0.36  0.940  359.91  358.320
7011  162771993.0     0.83 -0.790     -0.33  0.880  359.91  358.400

[7012 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_nofault = pd.read_csv("ESC_0.csv")
esc_0_m3_nofault = esc_0_m3_nofault.astype("float64")
esc_0_m3_nofault = esc_0_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_nofault = esc_0_m3_nofault[((esc_0_m3_nofault['TimeUS'] >= min) & (esc_0_m3_nofault['TimeUS'] <= max))]
esc_0_m3_nofault=esc_0_m3_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_nofault = pd.read_csv("ESC_1.csv")
esc_1_m3_nofault = esc_1_m3_nofault.astype("float64")
esc_1_m3_nofault = esc_1_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_nofault = esc_1_m3_nofault[((esc_1_m3_nofault['TimeUS'] >= min) & (esc_1_m3_nofault['TimeUS'] <= max))]
esc_1_m3_nofault=esc_1_m3_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_nofault = pd.read_csv("ESC_2.csv")
esc_2_m3_nofault = esc_2_m3_nofault.astype("float64")
esc_2_m3_nofault = esc_2_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_nofault = esc_2_m3_nofault[((esc_2_m3_nofault['TimeUS'] >= min) & (esc_2_m3_nofault['TimeUS'] <= max))]
esc_2_m3_nofault=esc_2_m3_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_nofault = pd.read_csv("ESC_3.csv")
esc_3_m3_nofault = esc_3_m3_nofault.astype("float64")
esc_3_m3_nofault = esc_3_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_nofault = esc_3_m3_nofault[((esc_3_m3_nofault['TimeUS'] >= min) & (esc_3_m3_nofault['TimeUS'] <= max))]
esc_3_m3_nofault=esc_3_m3_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_nofault = pd.read_csv("ESC_4.csv")
esc_4_m3_nofault = esc_4_m3_nofault.astype("float64")
esc_4_m3_nofault = esc_4_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_nofault = esc_4_m3_nofault[((esc_4_m3_nofault['TimeUS'] >= min) & (esc_4_m3_nofault['TimeUS'] <= max))]
esc_4_m3_nofault=esc_4_m3_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_nofault = pd.read_csv("ESC_5.csv")
esc_5_m3_nofault = esc_5_m3_nofault.astype("float64")
esc_5_m3_nofault = esc_5_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_nofault = esc_5_m3_nofault[((esc_5_m3_nofault['TimeUS'] >= min) & (esc_5_m3_nofault['TimeUS'] <= max))]
esc_5_m3_nofault=esc_5_m3_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m3_nofault = pd.read_csv("ESC_0.csv")\nesc_0_m3_nofault = esc_0_m3_nofault.astype("float64")\nesc_0_m3_nofault = esc_0_m3_nofault.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m3_nofault = esc_0_m3_nofault[((esc_0_m3_nofault[\'TimeUS\'] >= min) & (esc_0_m3_nofault[\'TimeUS\'] <= max))]\nesc_0_m3_nofault=esc_0_m3_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m3_nofault = pd.read_csv("ESC_1.csv")\nesc_1_m3_nofault = esc_1_m3_nofault.astype("float64")\nesc_1_m3_nofault = esc_1_m3_nofault.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m3_nofault = esc_1_m3_nofault[((esc_1_m3_nofault[\'TimeUS\'] >= min) & (esc_1_m3_nofault[\'TimeUS\'] <= max))]\nesc_1_m3_nofault=esc_1_m3_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\n

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m3_nofault = pd.read_csv("IMU_0.csv")
imu_0_m3_nofault = imu_0_m3_nofault.astype("float64")
imu_0_m3_nofault = imu_0_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_nofault = imu_0_m3_nofault[((imu_0_m3_nofault['TimeUS'] >= min) & (imu_0_m3_nofault['TimeUS'] <= max))]

imu_1_m3_nofault = pd.read_csv("IMU_1.csv")
imu_1_m3_nofault = imu_1_m3_nofault.astype("float64")
imu_1_m3_nofault = imu_1_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_nofault = imu_1_m3_nofault[((imu_1_m3_nofault['TimeUS'] >= min) & (imu_1_m3_nofault['TimeUS'] <= max))]

imu_2_m3_nofault = pd.read_csv("IMU_2.csv")
imu_1_m3_nofault = imu_1_m3_nofault.astype("float64")
imu_2_m3_nofault = imu_2_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_nofault = imu_2_m3_nofault[((imu_2_m3_nofault['TimeUS'] >= min) & (imu_2_m3_nofault['TimeUS'] <= max))]

imu_m3_nofault = pd.concat((imu_0_m3_nofault, imu_1_m3_nofault, imu_2_m3_nofault))
imu_m3_nofault=imu_m3_nofault.groupby(imu_m3_nofault.TimeUS, as_index=False).mean()

display(imu_m3_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,85325609.0,0.028868,0.067373,0.039493,-0.235401,-0.155897,-9.885922
1,85328405.0,0.004214,0.074477,0.043752,-0.152127,-0.234340,-9.779253
2,85333916.0,-0.049977,-0.096769,-0.019975,-0.172505,-0.181171,-9.579307
3,85339625.0,-0.007887,0.045497,0.027620,-0.177058,-0.247625,-10.115180
4,85342546.0,-0.053252,0.030805,0.026912,-0.189411,-0.292507,-9.954394
...,...,...,...,...,...,...,...
26643,162760938.0,0.107311,-0.070561,0.103588,1.213829,1.442536,-12.046240
26644,162763425.0,0.251781,-0.224217,0.087508,1.249439,1.487485,-12.493687
26645,162766175.0,0.155702,-0.123752,0.096069,0.973104,1.483165,-12.692430
26646,162768911.0,0.058944,-0.002178,0.118539,1.109990,1.748380,-12.772627


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m3_nofault = pd.merge_ordered(imu_m3_nofault,att_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m3_nofault = pd.merge_ordered(m3_nofault, esc_0_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_1_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_2_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_3_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_4_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_5_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")'''
m3_nofault = pd.merge_ordered(m3_nofault, rcou_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m3_nofault["TimeUS"] = m3_nofault["TimeUS"] - m3_nofault.iloc[0]["TimeUS"]
m3_nofault["TimeUS"] = m3_nofault["TimeUS"].astype(int)
m3_nofault["Guasto"] = guasto

In [13]:
last_cell = m3_nofault.iloc[-1, m3_nofault.columns.get_loc('TimeUS')]
print(last_cell)

77448804


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
27104  77436128
27105  77438985
27106  77441842
27107  77444699
27108  77447556

[27109 rows x 1 columns]


In [16]:
m3_nofault_final = pd.merge_ordered(m3_nofault,df)
m3_nofault_final=m3_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m3_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.028868  0.067373  0.039493 -0.235401 -0.155897  -9.885922   
1          1770  0.028868  0.067373  0.039493 -0.235401 -0.155897  -9.885922   
2          2857  0.028868  0.067373  0.039493 -0.235401 -0.155897  -9.885922   
3          4566  0.004214  0.074477  0.043752 -0.152127 -0.234340  -9.779253   
4          5714  0.004214  0.074477  0.043752 -0.152127 -0.234340  -9.779253   
...         ...       ...       ...       ...       ...       ...        ...   
67371  77445072  0.058944 -0.002178  0.118539  1.109990  1.748380 -12.772627   
67372  77447556  0.058944 -0.002178  0.118539  1.109990  1.748380 -12.772627   
67373  77447761  0.124930 -0.054047  0.129580  1.373738  2.068923 -12.852467   
67374  77448154  0.124930 -0.054047  0.129580  1.373738  2.068923 -12.852467   
67375  77448804  0.124930 -0.054047  0.129580  1.373738  2.068923 -12.852467   

       DesRoll  Roll  DesPitch  Pitch  

In [17]:
m3_nofault_final = m3_nofault_final[m3_nofault_final.TimeUS.isin(to_be)]

In [18]:
m3_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.028868,0.067373,0.039493,-0.235401,-0.155897,-9.885922,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
2,2857,0.028868,0.067373,0.039493,-0.235401,-0.155897,-9.885922,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
4,5714,0.004214,0.074477,0.043752,-0.152127,-0.234340,-9.779253,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
5,8571,0.004214,0.074477,0.043752,-0.152127,-0.234340,-9.779253,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
8,11428,-0.049977,-0.096769,-0.019975,-0.172505,-0.181171,-9.579307,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67362,77436128,0.030334,0.002761,0.108563,0.819613,1.262616,-11.889360,0.93,-0.88,-0.38,0.94,359.91,358.325,1620.0,1423.0,1544.0,1513.0,1512.0,1545.0,0.0
67366,77438985,0.107311,-0.070561,0.103588,1.213829,1.442536,-12.046240,0.89,-0.91,-0.36,0.94,359.91,358.320,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
67368,77441842,0.251781,-0.224217,0.087508,1.249439,1.487485,-12.493687,0.89,-0.91,-0.36,0.94,359.91,358.320,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
67370,77444699,0.155702,-0.123752,0.096069,0.973104,1.483165,-12.692430,0.89,-0.91,-0.36,0.94,359.91,358.320,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0


In [19]:
from datetime import timedelta
m3_nofault_final=m3_nofault_final.reset_index(drop=True)
m3_nofault_final['TimeUS'] = pd.to_datetime(m3_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.028868,0.067373,0.039493,-0.235401,-0.155897,-9.885922,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
1,00:00:00.002857,0.028868,0.067373,0.039493,-0.235401,-0.155897,-9.885922,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
2,00:00:00.005714,0.004214,0.074477,0.043752,-0.152127,-0.234340,-9.779253,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
3,00:00:00.008571,0.004214,0.074477,0.043752,-0.152127,-0.234340,-9.779253,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
4,00:00:00.011428,-0.049977,-0.096769,-0.019975,-0.172505,-0.181171,-9.579307,0.12,1.98,0.24,-0.84,3.31,3.310,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27104,00:01:17.436128,0.030334,0.002761,0.108563,0.819613,1.262616,-11.889360,0.93,-0.88,-0.38,0.94,359.91,358.325,1620.0,1423.0,1544.0,1513.0,1512.0,1545.0,0.0
27105,00:01:17.438985,0.107311,-0.070561,0.103588,1.213829,1.442536,-12.046240,0.89,-0.91,-0.36,0.94,359.91,358.320,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
27106,00:01:17.441842,0.251781,-0.224217,0.087508,1.249439,1.487485,-12.493687,0.89,-0.91,-0.36,0.94,359.91,358.320,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
27107,00:01:17.444699,0.155702,-0.123752,0.096069,0.973104,1.483165,-12.692430,0.89,-0.91,-0.36,0.94,359.91,358.320,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_nofault_final)/350)):
        V1=m3_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_2496\4105447242.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_2496\4105447242.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.061089,0.012492,0.102574,0.127232,0.0,65.0,11.601795,5.462084,0.024582,0.005464,-0.211145,0.077798,92.0,88.0,2.250426,1.534058,0.101961,0.004979,-1.413486,0.123941,61.0,52.0,3.971012,0.559449,-0.223162,0.040052,-0.428907,0.299565,105.0,104.0,16.943049,13.640850,0.248230,0.109864,-0.662429,0.413726,106.0,102.0,10.502291,8.917739,-10.919005,0.693396,-1.270135,10.950620,0.0,103.0,163.300066,14.948866,0.869074,1.851773,-1.665134,1.613006,108.0,105.0,26.314655,4.061431,0.180328,0.512049,-1.674936,0.736959,117.0,115.0,11.209720,1.453092,5.292080,3.929744,-1.239235,5.650191,77.0,105.0,-1.136983,-1.691122,1600.555556,7749.796190,0.437637,1602.967811,77.0,79.0,984.722580,978.057567,1512.965812,3400.993114,0.148836,1514.086144,89.0,90.0,1192.184343,956.661400,1563.034188,5054.747399,-0.197984,1564.645717,98.0,96.0,548.100715,358.366206,1554.754986,4230.934082,0.883022,1556.111161,91.0,95.0,746.245187,626.147579,1543.886040,3823.809833,0.439637,1545.120390,96.0,99.0,1262.296045,1191.825701,1572.501425,6864.062141,0.225475,1574.676232,90.0,92.0,1228.105831,1059.131593,0
1,1.0,-0.060948,0.012463,0.106833,0.127052,0.0,60.0,11.711611,5.030551,0.023967,0.005581,-0.135853,0.078357,81.0,54.0,2.096530,1.959332,0.101997,0.004966,-1.408271,0.123915,68.0,59.0,3.962860,0.500779,-0.222524,0.040081,-0.437205,0.299139,104.0,103.0,15.306274,13.517553,0.248642,0.109610,-0.657941,0.413669,98.0,95.0,8.850675,8.678846,-10.917042,0.692777,-1.271626,10.948635,0.0,99.0,164.586569,15.066624,0.862969,1.859618,-1.668916,1.612157,109.0,106.0,25.730341,4.329898,0.180739,0.510650,-1.670675,0.736115,105.0,103.0,11.038437,1.604368,5.303196,3.962045,-1.241615,5.663451,76.0,104.0,-1.556557,-1.647600,1600.806818,7749.939782,0.445363,1603.218760,83.0,82.0,1180.895388,1050.067412,1512.795455,3401.519296,0.131476,1513.916095,98.0,99.0,1076.955235,910.892485,1563.241477,5055.471437,-0.190730,1564.853036,99.0,94.0,564.492204,423.694699,1554.659091,4222.117068,0.880384,1556.012536,100.0,104.0,647.683262,556.615935,1544.005682,3817.954384,0.449420,1545.238057,87.0,86.0,1361.845972,1199.943628,1572.511364,6844.541181,0.235213,1574.679995,87.0,89.0,1129.693461,1019.239632,0
2,2.0,-0.061041,0.012431,0.114914,0.126971,0.0,60.0,11.737267,4.714979,0.023606,0.005612,-0.152914,0.078440,94.0,90.0,2.023777,1.511028,0.102091,0.004955,-1.403175,0.123948,67.0,61.0,3.974524,0.472590,-0.221852,0.040126,-0.446439,0.298716,103.0,102.0,13.440664,13.065846,0.249322,0.109462,-0.657071,0.413901,93.0,91.0,9.034858,8.077582,-10.914812,0.692563,-1.274289,10.946402,0.0,93.0,165.955331,15.311906,0.856898,1.867345,-1.672675,1.611313,118.0,115.0,25.135361,4.587853,0.181147,0.509258,-1.666411,0.735275,109.0,107.0,10.870989,1.768129,5.314249,3.993917,-1.244438,5.676605,79.0,

In [23]:
path_file = path_file.replace(r"M3\NO_FAULT3", "")
os.chdir(path_file)
df_merged.to_csv('m3_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
